<a href="https://colab.research.google.com/github/Vikas-KM/machine-learning/blob/master/ML_Projects/Movie_Recommendation_using_Naive_Bayes_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation using Naive Bayes 

Articles Referred
- https://iopscience.iop.org/article/10.1088/1742-6596/1345/4/042042/pdf
  -  Liu Shuxian and Fang Sen 2019 J. Phys.: Conf. Ser. 1345 042042

Movielens Dataset Referred
- https://grouplens.org/datasets/movielens/

link - http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html

### About the dataset
- The dataset contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.
- Only movies with at least one rating or tag are included in the dataset. These movie ids are consistent with those used on the MovieLens web site (e.g., id 1 corresponds to the URL https://movielens.org/movies/1). Movie ids are consistent between ratings.csv, tags.csv, movies.csv, and links.csv (i.e., the same id refers to the same movie across these four data files).

### Ratings Data File Structure (ratings.csv)
- All ratings are contained in the file ratings.csv. Each line of this file after the header row represents one rating of one movie by one user, and has the following format:

- userId,movieId,rating,timestamp
The lines within this file are ordered first by userId, then, within user, by movieId.

- Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

- Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.


### Tags Data File Structure (tags.csv)
- All tags are contained in the file tags.csv. Each line of this file after the header row represents one tag applied to one movie by one user, and has the following format:

- userId,movieId,tag,timestamp
- The lines within this file are ordered first by userId, then, within user, by movieId.

- Tags are user-generated metadata about movies. Each tag is typically a single word or short phrase. The meaning, value, and purpose of a particular tag is determined by each user.

- Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.

### Movies Data File Structure (movies.csv)
- Movie information is contained in the file movies.csv. Each line of this file after the header row represents one movie, and has the following format:

- movieId,title,genres
- Movie titles are entered manually or imported from https://www.themoviedb.org/, and include the year of release in parentheses. Errors and inconsistencies may exist in these titles.

- Genres are a pipe-separated list, and are selected from the following:

  - Action
  - Adventure
  - Animation
  - Children's
  - Comedy
  - Crime
  - Documentary
  - Drama
  - Fantasy
  - Film-Noir
  - Horror
  - Musical
  - Mystery
  - Romance
  - Sci-Fi
  - Thriller
  - War
  - Western
  - (no genres listed)

In [1]:
# importing some standard libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

In [15]:
# Mounting the google drive to download the dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Lets Import the movielens dataset
# using small dataset to work on later we will shift to 25M dataset

! ls /content/drive/MyDrive/ML_Datasets/movielens_small

links.csv  movies.csv  ratings.csv  README.txt	tags.csv


In [8]:
df_movies = pd.read_csv('/content/drive/MyDrive/ML_Datasets/movielens_small/movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
# shape of the movies dataset
df_movies.shape

(9742, 3)

In [18]:
# columns in the movies dataset
df_movies.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [19]:
# importing the ratings dataset
df_ratings = pd.read_csv('/content/drive/MyDrive/ML_Datasets/movielens_small/ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [20]:
# shape of the ratings dataset
df_ratings.shape

(100836, 4)

In [21]:
# columns in the ratings dataset
df_ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

## References Used:
- https://www.analyticsvidhya.com/blog/2020/11/create-your-own-movie-movie-recommendation-system/
